In [7]:
#Библиотеки
import pandas as pd
import numpy as np
import warnings
import json
import os
warnings.filterwarnings('ignore')

#Функция изъятия первых n слов
def cut_words(df, col, n):
    new = df[col].str.split(' ')
    new = new.str[0:n]
    new = new.apply(lambda x: ' '.join(x))
    return new

class html_formatter():
    def expandable(self, level, title, row, parent, expand, style, collapse):
        row_append = f'<tr class="treegrid-{level}{parent} {expand}" id="node-1-1" style="{style}">' \
                     f'<td>' \
                     f'<span {collapse} </span>'\
                     f'{title}' \
                     f'</td>' \
                     f'<td>{row[0]}</td>' \
                     f'<td>{row[1]}</td>' \
                     f'<td>{row[2]}</td>' \
                     f'<td>{row[3]}</td>' \
                     f'</tr>'
        return row_append
html_formatter = html_formatter()

#Открытие json
with open('./data.json') as fp:
    data = json.load(fp)

#1-ый уровень
fields = ['TITLE', 'ITEMS']
table = pd.json_normalize(data,max_level=1)[fields]
table = table.explode('ITEMS', ignore_index=True)
table = pd.concat([table, table['ITEMS'].apply(pd.Series)], axis=1)
fields = ['TITLE', 'title', 'ITEMS', 'STATUS_RESULT']
table = table[fields]
col_replace = {'TITLE': 'project', 'title': 'stage',
               'ITEMS': 'items', 'STATUS_RESULT': 'status_stage'}
table.rename(columns = col_replace, inplace = True)
table = table.iloc[:, np.r_[0,2,4,5]]
table = table.explode('items', ignore_index=True)
table = pd.concat([table, table['items'].apply(pd.Series)], axis=1)

#2-ой уровень
fields = ['project', 'stage', 'status_stage', 'title',
          'RESPONSOBLE_FIO', 'DURATION', 'BEGIN_DATE_FORMAT',
          'CLOSE_DATE_FORMAT', 'STATUS_RESULT', 'ITEMS', 'TASKS']
table = table[fields]
col_replace = {'title':'function', 'RESPONSOBLE_FIO': 'role_function',
               'DURATION': 'duration_function', 'BEGIN_DATE_FORMAT': 'start_function',
               'CLOSE_DATE_FORMAT': 'finish_function', 'STATUS_RESULT': 'status_function',
               'ITEMS': 'items', 'TASKS': 'tasks'}
table.rename(columns = col_replace, inplace = True)

#3-й уровень
fields = ['project', 'stage', 'function', 'status_stage', 'role_function',
         'duration_function', 'start_function', 'finish_function', 'tasks']
table = table[fields]
table = table.explode('tasks', ignore_index=True)
table = pd.concat([table, table['tasks'].apply(pd.Series)], axis=1)

#Удаление лишних столбцов
fields.append('TITLE')
table = table[fields]
col_replace = {'TITLE': 'task',
               'status_stage': 'status'}
table.rename(columns = col_replace, inplace = True)
table = table.drop('tasks', axis = 1)

#Создание структуры таблицы
table['decomposition_1'] = np.where(table['stage']==' ',
                                    'Задачи договора',
                                    'Этапы договора')
table['document'] = np.where(cut_words(table, 'function', 1).str[:-1].str.isnumeric()==True,
                             table['function'],
                             ' ')
table['function'] = np.where(table['document'] != ' ',
                             ' ',
                             table['function'])
table['decomposition_2'] = np.where(table['function']==' ',
                                    np.where(table['document']!=' ',
                                             'Документооборот',
                                             table['task']),
                                    'Функции')
table['task'] = np.where(table['task'] == table['decomposition_2'],
                         ' ',
                         table['task'])
table['decomposition_3'] = np.where(table['decomposition_2'] == 'Функции',
                                    table['function'],
                                    np.where(table['decomposition_2'] == 'Документооборот',
                                             table['document'],
                                             ' '))
table['stage'] = np.where(table['decomposition_1']=='Задачи договора',
                          table['decomposition_2'],
                          table['stage'])
table['decomposition_2'] = np.where(table['decomposition_2']==table['stage'],
                                    ' ',
                                    table['decomposition_2'])

#Добавление иерархии
col_order = ['project', 'decomposition_1', 'stage',
             'decomposition_2', 'decomposition_3', 'task',
             'role_function', 'duration_function', 'start_function',
             'finish_function']
table = table[col_order]
for i in range(5):
    level = list(set(map(tuple,table.iloc[:,:(1+i)].values.tolist())))
    length = table.shape[1] - len(level[0])
    for j in level:
        row = list(j)
        table = table.append(pd.Series(row + [' ']*length,
                             index=list(table.columns)),
                             ignore_index=True)

#Составление сводной таблицы
pivot = table.pivot_table(index = col_order[:6],
                          values = col_order[6:],
                          aggfunc ='sum')
pivot = pivot[~pivot.index.duplicated(keep='first')]
pivot = pivot[['duration_function', 'role_function',
               'start_function', 'finish_function']]
pivot['role_function'] = np.where((pivot['role_function'] == ' ') &
                                  (pivot['start_function'] != ' '),
                      '-',
                      pivot['role_function'])

#Сброс индекса
pivot.to_excel('./data.xlsx')
pivot = pd.read_excel('./data.xlsx').replace(np.nan, ' ')
os.remove('./data.xlsx')

#Добавление отступов
expands = list(pivot.columns)[:6]
for k, col in enumerate(expands):
    pivot[col] = np.where(pivot[col] == ' ',
                                        ' ',
                                        '__'*k + pivot[col].astype(str))
#html-конвертация
header = '<!doctype html>' \
         '<html>' \
         '<head>' \
         '<meta charset="utf-8">' \
         '<title>Расписание</title>' \
         '<link rel="stylesheet" href="jquery.treegrid.css">' \
         '</head>' \
         '<body style>' \
         '<table>'\
         '<tbody>'\
         '<tr>'\
         '<td>'\
         '<table id="tree-1" border="1" cellpadding="0" cellspacing="0">' \
         '<tbody>' \
         '<tr id="tree-head-1">' \
         f'<th>Иерархия</th>' \
         f'<th>{list(pivot.columns)[6]}</th>' \
         f'<th>{list(pivot.columns)[7]}</th>' \
         f'<th>{list(pivot.columns)[8]}</th>' \
         f'<th>{list(pivot.columns)[9]}</th>' \
         '</tr>'
footer = '</tbody>'\
         '</table>' \
         '</td>'\
         '</tr>'\
         '</tbody>'\
         '</table>'\
         '<script src="http://ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>'\
         '<script src="jquery.treegrid.min.js"></script>'\
         '<script src="jquery.cookie.js"></script>'\
         '<script src="test.js"></script>'\
         '<script>'\
            'var globalCounter = 0;'\
            'var saveStateName = "state-save-4";'\
            '$("#tree-1").treegrid({initialState: "collapsed"});'\
            '$("#tree-2").treegrid({treeColumn: 1, initialState: "expanded", "saveState": true, "saveStateName": saveStateName});'\
            '$("#tree-3").treegrid({initialState: "collapsed"});'\
            '$("#tree-4").treegrid({onChange: function() {'\
                    'globalCounter++;'\
                '}, onCollapse: function() {'\
                    'globalCounter++;'\
                '}, onExpand: function() {'\
                    'globalCounter++;'\
                '}});'\
         '</script>'\
         '</body>' \
         '</html>'
body = ''
length = pivot.shape[0]
for i in range(length):
   row_ = list(pivot.iloc[i, :])
   for k, pos in enumerate(row_):
       if pos!= ' ':
           level = k + 1
           break;
   title = [x for x in row_ if x.isspace() == False]
   if (level != 6) & (len(title) == 1):
       row = [' ']*(5-len(title))
   else:
       row = title[1:] + [' ']*(5-len(title))
       title = [title[0]]
   if level == 1:
       parent = ''
       expand = 'treegrid-collapsed'
       style=''
       collapse = 'class="treegrid-expander treegrid-expander-collapsed">'
   else:
       parent = f' treegrid-parent-{level-1}'
       expand = 'treegrid-expanded'
       style = 'display: none;'
       collapse = 'class="treegrid-expander treegrid-expander-expanded">'
   body += html_formatter.expandable(level, title[0], row,
                                     parent, expand, style,
                                     collapse)
   html_doc = header + body + footer

#Формирование html-таблицы
with open('./table.html', 'w', newline='', encoding="utf-8") as file:
    file.write(html_doc)